## COMPSCI 762 Assignment 3

### Report Section

The goal of this programming project is to use a Naive Bayes algorithm to achieve high predictive accuracy on the test data. The package I used in this project is only `numpy`. The first step used `numpy` to load in the training data "trg.csv" and I checked there are no missing values in this data set, and all the abstracts are at lower case letters and no punctuation, so there is no need to do the preprocessing with the data. Linking to the Naive Bayes algorithm, the probability of an abstract in a class is proportional to the probability of that class multiplied by the conditional probabilities of each word that appeared in that abstract if that word is in the count. Therefore, I tried to obtain an attribute probability representation of each word in each class. The main idea is to split each 'abstract' into words and calculate the conditional probabilities of each word that would appear in each 'class' and the probability of 'classes'.

For the model building, I first calculated the probabilities of each class and noticed that *class B* and *class C* take the majority of all the class labels. Therefore, I build a null model by randomly choose the class label from *class B* and *class C* as the predicted value and expect to have an accuracy of around 50%. 

For the running time efficiency, I first chose the 1000 words with the most occurrence frequency in the data. Then I write a standard naive base algorithm to multiple all the probabilities together. Based on the cases that there would be zero counts of a word in an abstract, which would lead to a zero conditional probability and results in a zero Naive Bayes probability; and multiplication of a lot of probabilities may lead to a minimal value with a lot of zero digits, which may cause some problems in the final answers. Therefore, I write an improved naive base algorithm that calculates conditional probability by adding a Laplace smoothing of 1 to the denominator and the number of unique words to the numerator and then using log() to add up the probabilities. Finally, in order to place less weight on very common words and more weight on less common word, I used the TF-IDF to replace the word frequency value with the multiplication of the word frequency and the log of the inverse document frequency to extend the improved naive base. Therefore, I have a total of four models to test the performance.

A cross-validation function splits the training data "trgarr" into a training set and a validation set to test the performance of the different model and different parameters. The training set is used to choose the words and calculate the conditional probability of each word in each class and the class probability. Then the validation set is used to predict the class for each abstract based on the algorithm built from the training set and validate the accuracy with the actual class. I chose the 5-folds cross-validation for time efficiency. Firstly, I used 5-folds cross-validation to test the training set without deleting the stop words. I ran cross-validation for the null model and the standard naive Bayes model with 1000 best words and the standard naive Bayes model with all the words. The accuracy score of the null model is 47.3% which means I can always have at least around 47.3% accuracy on the prediction, and there can be further improvement by using the naive Bayes algorithm. As the results shown in Table 1, the accuracy score of the standard naive Bayes dropped rapidly from 0.899 to 0.2045 by using all the words, it would be because many words do not occur in the abstract and lead to a zero probability, which affects the prediction. Thus it is important to add the Laplace smoothing and using log() to add up the probabilities. Then I removed the stop words such as "the", "of", which are useless and run the cross-validation for the standard naive Bayes algorithm, improved naive Bayes algorithm with smoothing and improved naive Bayes with TF-IDF with 1000 best words and the improved naive Bayes with TF-IDF with 15000 words. As shown in Table 2, the accuracy score of the standard naive Bayes has increased by 0.03, which means removing the stop words helped a lot to the prediction. The accuracy scores of the improved naive Bayes with smoothing and with TF-IDF are around 90% which are lower than the standard naive Bayes. However, the standard naive Bayes is sensitive to the choice of the number of words as it will drop down rapidly if the number of best words increases. As a result, for the improved naive Bayes with TF-IDF with 15000 best words, the accuracy score increases rapidly with about 5% accuracy. The number of the best words should also be a parameter that would affect the prediction accuracy. Then I tried to use cross-validation to choose the number of best words that result in the highest accuracy. Considering the running time, I ran the cross-validation with 5000 to 15000 number of best words with step size 1000. Then I find the **10000** number of best words give the highest accuracy score, and then the accuracy score goes down. (The results are in Table 3)

I also tested the accuracy score on the whole training data for the three naive Bayes models with the results shown in Table 4. By choosing 10000 best words, the accuracy scores for the improved naive Bayes with smoothing and the improved naive Bayes with TF-IDF are similar. Therefore, I tried both the improved naive Bayes without TF-IDF and the improved naive Bayes with TF-IDF with 10000 best words to train the whole training data "trg.csv" and apply the probabilities and algorithm to predict the test data "tst.csv". Then I uploaded them onto Kaggle and got scores of 0.9533 for both of the algorithms. I think this is because the main issue of this training set is the class imbalance, but TF-IDF is not a method to deal with it and leads to no significant improvement on the test data. The naive Bayes model can be further extended by counting a class weight.

  

| Table 1    | Null Model | Standard NB | Standard NB |
| :--------: | :--------: | :---------: | :---------: |
|no. of words|    1000    |     1000    |     All     |
| score      |   0.4728   |  **0.8997** |    0.2045   |


|   Table 2  | Standard NB | INB with Smoothing | INB with TF-IDF | INB with TF-IDF |
| :--------: | :---------: | :------------------------: | :---------------------: | :---------------------: |
|no. of words|     1000    |            1000            |           1000          |           15000         |   
| score      |    0.9208   |           0.9048           |          0.90025        |        **0.9518**       |


Table 3

|no. of words|  5000  |  6000  |  7000  |  8000  |  9000  |   10000  |  11000 |  12000 |  13000 |  14000 |  15000 |
| :--------: | :----: | :----: | :----: | :----: | :----: | :------: | :----: | :----: | :----: | :----: | :----: | 
| score      | 0.9445 | 0.9468 | 0.9483 | 0.9497 | 0.9528 |**0.9533**| 0.9530 | 0.9520 | 0.9520 | 0.9515 | 0.9518 |


|  Table 4   | Standard NB | Improved NB with Smoothing | Improved NB with TF-IDF |
| :--------: | :---------: | :------------------------: | :---------------------: |
|no. of words|    10000    |            10000           |           10000         |
| score      |    0.5895   |          **0.983**         |          0.98075        |

### Code Section

In [1]:
import numpy as np

#### Data Sorting

In [2]:
# count occurrence frequency of all words in a text
def count_word(d,text):
    for word in text[1:-1].split(): #remove the "" at the begining and end of the string and split the text into separate words
        if word.isdigit() == False: # remove integer values in the abstract
            if len(word) >1: # remove words with length less than 1
                if word not in d.keys(): #or word+'s' not in word_dict.keys() or word+'es' not in word_dict.keys():
                    d[word] = 1
                else:
                    d[word] += 1
    return d

# remove the stop words and sort the words with descending occurrence frequency
def sorted_words(abstracts):
    word_dict = {}
    for text in abstracts:
        # building a dictionary to store words appear in the abstract
        word_dict=count_word(word_dict,text)
    
    # sort the words occur frequency into descending order
    sorted_dict = dict(sorted(word_dict.items(), key=lambda item:item[1], reverse=True))
    return sorted_dict

def remove_stop_words(d):
    stop_words = ['the','of','and','in','to','is','for','are','was','by','as','an','we','has','be','been','that',
                  'other','on','at','or','not','also','its','between','contains','but','all','it','with','from',
                  'both','may','most','than','their','were','this','which','have','these','only','known','more',
                  'here','using','those','into','no','many','high','within','large','each','there','however','approximately',
                 'some','can','such','our','although','one','two','three','four','well','during','had','about','any',
                 'they','show','shows','shown','thus','five','could','when','same','very','while','whereas','ii','iii',
                 'under','so','over','almost','being','after','whole','then','even','yet','via','do','will','six',
                  'seven','eight','nine','ten','iv','vi','upon','did','therefore','whose','them','having','able','does',
                 'still']

    for word in stop_words:
        if word in d:
            del d[word]
    
    return d

# count the words occurrence frequency in each class
def words_frequency(words, abstracts, labels):
#def conditional_prob (words, abstracts, labels):
    # initialise dictionaries for each of four domains
    A_dict = {}
    B_dict = {}
    E_dict = {}
    V_dict = {}

    for word in words:
        A_dict[word] = 0
        B_dict[word] = 0
        E_dict[word] = 0
        V_dict[word] = 0

    # count the words' occurrence frequency in the abstract to each domain.
    for i in range(len(abstracts)):
        for word in abstracts[i][1:-1].split():
            if word in words:
                #occurence frequency of each word in class A.
                if labels[i] == 'A':
                    A_dict[word] += 1
                #occurence frequency of each word in class B.
                if labels[i] == 'B':
                    B_dict[word] += 1
                #occurence frequency of each word in class E.
                if labels[i] == 'E':
                    E_dict[word] += 1
                #occurence frequency of each word in class V.
                if labels[i] == 'V':
                    V_dict[word] += 1

    return [A_dict,B_dict,E_dict,V_dict]

def probability (d):
    total = sum(d.values()) # total word occurrence frequency
    unique = len(d) # number of unique words
    for key, value in d.items():
        # laplace smoothing
        d[key] = np.log((value + 1)/(total + unique))
    return d

# calculate the conditional probability
def conditional_prob(words, abstracts, labels):
    frequency = words_frequency(words, abstracts, labels)
    A = probability(frequency[0])
    B = probability(frequency[1])
    E = probability(frequency[2])
    V = probability(frequency[3])
    
    return [A,B,E,V]

# calculate the class probability
def class_prob (label):
    A_class = sum(label=='A') # occurence of class A
    B_class = sum(label=='B') # occurence of class B
    E_class = sum(label=='E') # occurence of class E
    V_class = sum(label=='V') # occurence of class V
    total = A_class + B_class + E_class + V_class

    # probability of each class
    return [np.log(A_class/total), np.log(B_class/total), np.log(E_class/total), np.log(V_class/total)]

### Null Model

In [3]:
#null model
def null_model(X):
    result = []
    for i in range(len(X)):
        predicted = np.random.choice(['B','E'])
        result.append(predicted)
    return result

### Standard Naive Bayes

In [4]:
def standard_probability(d):
    total = sum(d.values()) # total word occurrence frequency
    unique = len(d) # number of unique words
    for key, value in d.items():
        d[key] = value/total
    return d

# calculate the conditional probability
def standard_cond_prob(words, abstracts, labels):
    frequency = words_frequency(words, abstracts, labels)
    A = standard_probability(frequency[0])
    B = standard_probability(frequency[1])
    E = standard_probability(frequency[2])
    V = standard_probability(frequency[3])
    
    return [A,B,E,V]

# calculate the class probability
def standard_class_prob (label):
    A_class = sum(label=='A') # occurence of class A
    B_class = sum(label=='B') # occurence of class B
    E_class = sum(label=='E') # occurence of class E
    V_class = sum(label=='V') # occurence of class V
    total = A_class + B_class + E_class + V_class

    # probability of each class
    return [A_class/total, B_class/total, E_class/total, V_class/total]

def standard_predict(abstracts,cond_prob,class_prob,type_words):
    result=[]
    
    # get the class probability
    p_A = class_prob[0]
    p_B = class_prob[1]
    p_E = class_prob[2]
    p_V = class_prob[3]
    
    # get the conditional probability
    A_dict = cond_prob[0]
    B_dict = cond_prob[1]
    E_dict = cond_prob[2]
    V_dict = cond_prob[3]
    
    for text in abstracts:
        score_dict={}
        score_dict['A']=p_A
        score_dict['B']=p_B
        score_dict['E']=p_E
        score_dict['V']=p_V
        
        d = {}
        counts = count_word(d,text)
        
        # naive bayes algorithm with log value
        for word in counts.keys():
            if word in type_words:
                score_dict['A'] *= A_dict[word]
                score_dict['B'] *= B_dict[word]
                score_dict['E'] *= E_dict[word]
                score_dict['V'] *= V_dict[word]
        
        value = list(score_dict.values())
        # find the maximum probability is the predicted class
        i = value.index(max(value))
        key = list(score_dict.keys())
        res = key[i]
        result.append(res)
        
    return result

### TF-IDF 

In [5]:
def count_abstract (words, abstracts):
    d = {}
    for word in words:
        d[word] = 0
        for text in abstracts[1:-1]:
            if text.find(word):
                d[word] += 1
    return d

# calculate the idf
def idf(words, abstracts, labels):
    length = len(abstracts)
    word_dict = count_abstract(words,abstracts)

    for i in word_dict.keys():
        word_dict[i] = np.log(length/word_dict[i]+1)   #+1
    return word_dict

# calculate the TF-IDF value for each word in each class
def tfidf(words, abstracts, labels):
    tf = words_frequency(words, abstracts, labels)
    idfd = idf(words, abstracts, labels)
    
    A = tf[0]
    B = tf[1]
    E = tf[2]
    V = tf[3]
    
    A_tfidf = {}
    B_tfidf = {}
    E_tfidf = {}
    V_tfidf = {}
    
    # replace frequency with frequency * IDF
    for word in idfd:
        A_tfidf[word] = A[word]*idfd[word]
        B_tfidf[word] = B[word]*idfd[word]
        E_tfidf[word] = E[word]*idfd[word]
        V_tfidf[word] = V[word]*idfd[word]
    
    A_prob = probability(A_tfidf)
    B_prob = probability(B_tfidf)
    E_prob = probability(E_tfidf)
    V_prob = probability(V_tfidf)
    
    return [A_prob,B_prob,E_prob,V_prob]

### Cross Validation

In [6]:
def predict(abstracts,cond_prob,class_prob,type_words):
    result=[]
    
    # get the class probability
    p_A = class_prob[0]
    p_B = class_prob[1]
    p_E = class_prob[2]
    p_V = class_prob[3]
    
    # get the conditional probability
    A_dict = cond_prob[0]
    B_dict = cond_prob[1]
    E_dict = cond_prob[2]
    V_dict = cond_prob[3]
    
    for text in abstracts:
        score_dict={}
        score_dict['A']=p_A
        score_dict['B']=p_B
        score_dict['E']=p_E
        score_dict['V']=p_V
        
        words = text[1:-1].split()
    
        # naive bayes algorithm with log value
        for word in words:
            if word in type_words:
                score_dict['A'] += A_dict[word]
                score_dict['B'] += B_dict[word]
                score_dict['E'] += E_dict[word]
                score_dict['V'] += V_dict[word]
        
        value = list(score_dict.values())
        # find the maximum probability is the predicted class
        i = value.index(max(value))
        key = list(score_dict.keys())
        res = key[i]
        result.append(res)
        
    return result

def standard_algorithm(Xtrain,Xvalidation,Ytrain,type_words):
    cond_prob = standard_cond_prob(type_words, Xtrain, Ytrain)
    label_prob = standard_class_prob(Ytrain)
    
    predicted = standard_predict(Xvalidation,cond_prob,label_prob,type_words)
    return predicted
                       
def algorithm(Xtrain,Xvalidation,Ytrain,type_words):
    cond_prob = conditional_prob(type_words, Xtrain, Ytrain)
    label_prob = class_prob(Ytrain)
    
    predicted = predict(Xvalidation,cond_prob,label_prob,type_words)
    return predicted

def tfidf_algorithm(Xtrain,Xvalidation,Ytrain,type_words):
    cond_prob = tfidf(type_words, Xtrain, Ytrain)
    label_prob = class_prob(Ytrain)
    
    predicted = predict(Xvalidation,cond_prob,label_prob,type_words)
    return predicted

def accuracy(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct/len(actual)

def cross_validation(data,n,type_words,model):
    fold_size = int(len(data)/n) # number of rows in each fold
    ntrain = int(fold_size*(n-1)) # number of rows in training set
    ntest = int(fold_size) # number of rows in testing set
    scores = []
    for i in range (n):
        start = fold_size * i
        
        # train test data split for X value
        Xvalidation = data['abstract'][start:start+ntest]
        Xtrain = np.append(data['abstract'][:start],data['abstract'][start+ntest:])
        
        # train test data split for Y value
        Yvalidation = data['class'][start:start+ntest] 
        Ytrain = np.append(data['class'][:start],data['class'][start+ntest:])
        
        if model == 0:
            predicted = null_model(Xtrain)
        elif model == 1:
            predicted = standard_algorithm(Xtrain,Xvalidation,Ytrain,type_words)
        elif model == 2:
            predicted = algorithm(Xtrain,Xvalidation,Ytrain,type_words)
        elif model == 3:
            predicted = tfidf_algorithm(Xtrain,Xvalidation,Ytrain,type_words)
        
        score = accuracy(Yvalidation, predicted)
        scores.append(score)
    return scores 

### Load the Training Set

In [7]:
# setting the data type of each column
dt = np.dtype([('id', np.int32), ('class', 'object'), ('abstract', 'object')])

# using numpy to read the trg.csv in as array
trgarr = np.loadtxt("trg.csv",delimiter=",", skiprows=1, dtype=dt)

# check any missing value in the trgarr
lis = [item != item for item in trgarr]
print("number of missing value in the training set is:", sum(lis))

# look at the probability of each class occurs
label = standard_class_prob(trgarr['class'])
print("probability of class A: ", label[0])
print("probability of class B: ", label[1])
print("probability of class E: ", label[2])
print("probability of class V: ", label[3])

number of missing value in the training set is: 0
probability of class A:  0.032
probability of class B:  0.4005
probability of class E:  0.536
probability of class V:  0.0315


### Stop words not deleted

In [8]:
sorted_dict = sorted_words(trgarr['abstract'])

# list of all words
all_words = list(sorted_dict.keys())
# list of the 1000 most frequent occured words
freq_words = all_words[:1000]

> Using cross validation to test model

In [9]:
# The null model performance with 1000 best words
null = cross_validation(trgarr,5,freq_words,0)
print("The accuracy score for the null model is:", sum(null)/len(null))

# The standard naive bayes performance with 1000 best words
score = cross_validation(trgarr,5,freq_words,1)
print("The accuracy score for using the 1000 best words:", sum(score)/len(score))

# The standard naive bayes performance with all words
score = cross_validation(trgarr,5,all_words,1)
print("The accuracy score for using all words:", sum(score)/len(score)) 

The accuracy score for the null model is: 0.47275
The accuracy score for using the 1000 best words: 0.8997499999999998
The accuracy score for using all words: 0.2045


### Stop words deleted

In [10]:
clean_words = list(remove_stop_words(sorted_dict).keys())
freq_clean = clean_words[:1000]

> Using cross validation to test model

In [11]:
# The standard naive bayes performance with 1000 best words
score = cross_validation(trgarr,5,freq_clean,1)
print("Standard Naive Bayes with 1000 best words:", sum(score)/len(score))

# The improved naive bayes performance with 1000 best words
score = cross_validation(trgarr,5,freq_clean,2)
print("Improved Naive Bayes with 1000 best words:", sum(score)/len(score))

# The improved naive bayes with TF-IDF performance with 1000 best words
score = cross_validation(trgarr,5,freq_clean,3)
print("Improved Naive Bayes with TF-IDF with 1000 best words:", sum(score)/len(score))

# The improved naive bayes with TF-IDF performance with all words
score = cross_validation(trgarr,5,clean_words[:15000],3)
print("Improved Naive Bayes with TF-IDF with 15000 best words:", sum(score)/len(score))



Standard Naive Bayes with 1000 best words: 0.92075
Improved Naive Bayes with 1000 best words: 0.9047500000000002
Improved Naive Bayes with TF-IDF with 1000 best words: 0.90025
Improved Naive Bayes with TF-IDF with 15000 best words: 0.95175


In [12]:
# Test the accuracy score of naive bayes with TF-IDF performance with 5000 to 15000 best words with step size 1000
for i in range (5000,15001,1000):
    word_length = clean_words[:i]
    score = cross_validation(trgarr,5,word_length,3)
    print(i, sum(score)/len(score))

5000 0.9445
6000 0.9467500000000001
7000 0.94825
8000 0.9497499999999999
9000 0.95275
10000 0.95325
11000 0.9530000000000001
12000 0.952
13000 0.952
14000 0.9515
15000 0.95175


In [13]:
# accuracy score on full training set by using standard naive bayes with 10000 best words
cond_prob = standard_cond_prob(clean_words[:10000], trgarr['abstract'], trgarr['class'])
label_prob = standard_class_prob(trgarr['class'])
predicted = predict(trgarr['abstract'],cond_prob,label_prob,clean_words[:10000])
score = accuracy(trgarr['class'], predicted)
print("Standard Naive Bayes with 10000 best words on full training set:", score)

# accuracy score on full training set by using improved naive bayes with 1000 best words
cond_prob = conditional_prob(clean_words[:10000], trgarr['abstract'], trgarr['class'])
label_prob = class_prob(trgarr['class'])
predicted = predict(trgarr['abstract'],cond_prob,label_prob,clean_words[:10000])
score = accuracy(trgarr['class'], predicted)
print("Improved Naive Bayes with 10000 best words on full training set:", score)

# accuracy score on full training set by using improved naive bayes with TF-IDF with 1000 best words
cond_prob = tfidf(clean_words[:10000], trgarr['abstract'], trgarr['class'])
label_prob = class_prob(trgarr['class'])
predicted = predict(trgarr['abstract'],cond_prob,label_prob,clean_words[:10000])
score = accuracy(trgarr['class'], predicted)
print("Improved Naive Bayes with TF-IDF with 10000 best words on full training set:", score)

Standard Naive Bayes with 10000 best words on full training set: 0.5895
Improved Naive Bayes with 10000 best words on full training set: 0.983
Improved Naive Bayes with TF-IDF with 10000 best words on full training set: 0.98075


### Loading the Testing Set

In [14]:
# setting the data type of each column
dt = np.dtype([('id', np.int32), ('abstract', 'object')])

# using numpy to read the test data in as array
tstarr = np.loadtxt("tst.csv",delimiter=",", skiprows=1, dtype=dt)

### Predicting for the Testing Set

In [15]:
abstract = tstarr['abstract']
words_10000 = clean_words[:10000]

# without TF-IDF
cond_prob = conditional_prob(words_10000, trgarr['abstract'],trgarr['class'])
label_prob = class_prob(trgarr['class'])
    
result1 = predict(abstract,cond_prob,label_prob,words_10000)

In [16]:
# with TF-IDF
cond_prob = tfidf(words_10000,trgarr['abstract'],trgarr['class'])
label_prob = class_prob(trgarr['class'])

result2 = predict(abstract,cond_prob,label_prob,words_10000)

### Convert the output to CSV file

In [17]:
# without TF-IDF
empty_array1=np.empty((1000,0),object)

comb_arr1 = np.append(empty_array1,np.array([tstarr['id']]).transpose(),axis=1)
comb_arr1 = np.append(comb_arr1,np.array([result1]).transpose(),axis=1)
print(comb_arr1)

np.savetxt("submission1.csv", comb_arr1, delimiter=",", fmt = '% s', header='id,class', comments='')

# with TF-IDF
empty_array2=np.empty((1000,0),object)

comb_arr2 = np.append(empty_array2,np.array([tstarr['id']]).transpose(),axis=1)
comb_arr2 = np.append(comb_arr2,np.array([result2]).transpose(),axis=1)
print(comb_arr2)

np.savetxt("submission2.csv", comb_arr2, delimiter=",", fmt = '% s', header='id,class', comments='')

[[1 'B']
 [2 'E']
 [3 'E']
 ...
 [998 'E']
 [999 'A']
 [1000 'E']]
[[1 'B']
 [2 'E']
 [3 'E']
 ...
 [998 'E']
 [999 'A']
 [1000 'E']]
